In [1]:
# Local imports

# External imports
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import Model
import spektral as sp
import scipy as sc

In [2]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  0


In [3]:
# Load graph-MNIST dataset
mnist = sp.datasets.mnist.MNIST(p_flip=0.0, k=8)

In [22]:
# Prepare Ahat
A = mnist.a.todense()
A_tilde = A + np.eye(N=A.shape[0])
D_tilde = np.diagflat(A_tilde.sum(axis=1))
D_rt = sc.linalg.fractional_matrix_power(D_tilde, -1/2)
Ahat = D_rt @ A_tilde @ D_rt
Ahat = np.array(Ahat, dtype='float32')

In [41]:
# Prepare dataset
train_size = 40000
val_size = 20000
test_size = 10000
batch_size = 1024

# Define training, validation and test_data
x_train = np.array([mnist[i].x for i in range(0, train_size)], dtype='float32').reshape(train_size, 784)
y_train = np.array([mnist[i].y for i in range(0, train_size)], dtype='float32')

x_val = np.array([mnist[i].x for i in range(train_size, train_size+val_size)], dtype='float32').reshape(val_size, 784)
y_val = np.array([mnist[i].y for i in range(train_size, train_size+val_size)], dtype='float32')

x_test = np.array([mnist[i].x for i in range(train_size+val_size, train_size+val_size+test_size)], dtype='float32').reshape(test_size, 784)
y_test = np.array([mnist[i].y for i in range(train_size+val_size, train_size+val_size+test_size)], dtype='float32')

# (x_train, y_train), (x_test, y_test) = mnist.load_data()
# x_train, x_test = x_train / 255.0, x_test / 255.0

# # Add a channels dimension
# x_train = x_train[..., tf.newaxis].astype("float32")
# x_test = x_test[..., tf.newaxis].astype("float32")

# Add data formatting and preparation here
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(batch_size)

val_ds = test_ds = tf.data.Dataset.from_tensor_slices((x_val, y_val)).batch(batch_size)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)

In [43]:
class GCNLayer(tf.keras.layers.Layer):
    def __init__(self, conv_size, Ahat, act=tf.nn.relu):
        super(GCNLayer, self).__init__()
        self.Ahat = Ahat
        self.F = conv_size # Output is NxF - decide N at build time
        self.act = tf.nn.relu
    
    def build(self, input_shape):
        self.N = input_shape[0]
        self.C = input_shape[1]
        # Define the weight matrix for this layer - of size CxF
        self.kernel = self.add_weight('kernel', 
                                     shape=[self.C, self.F],
                                     dtype='float32')
        
    def call(self, X):
        # Perform forward pass, calculates Z
        b = tf.matmul(self.Ahat, tf.matmul(X, self.kernel))
        Z = self.act(b)
        return Z
    
class GCNModel(Model):
    def __init__(self, F, Ahat):
        super(GCNModel, self).__init__()
        self.F = F
        self.Ahat = Ahat
        self.gcn1 = GCNLayer(self.F, self.Ahat, act=tf.nn.relu)
        self.gcn2 = GCNLayer(self.F, self.Ahat, act=tf.nn.relu)
        self.add = tf.keras.layers.Add()
        self.d1 = tf.keras.layers.Dense(self.F, activation='relu')
        self.d2 = tf.keras.layers.Dense(1, activation='sigmoid')

    def call(self, X):
        N = X.shape[0]
        # 1. GCN layers
        O1 = self.gcn1(X)
        O2 = self.gcn2(O1)
        # 2. Aggegate nodal values by summing over nodes => NxF -> global vector of length F
        # Note that this is invariant under permutation of the graph!
        split_O = tf.split(O2, num_or_size_splits=N, axis=0)
        x = tf.keras.layers.Add()(split_O)
        # 3. Apply FCNN for classification
        x = self.d1(x)
        return self.d2(x)

# Create an instance of the model and define loss etc.
# Update these with real data!
F = 2
model = GCNModel(F, Ahat)
for graphs, labels in train_ds:
    print(graphs.shape)
    out = model(graphs)
    print(out)

(1024, 784)


InvalidArgumentError: Matrix size-incompatible: In[0]: [784,784], In[1]: [1024,2] [Op:MatMul]

In [30]:
# Loss & metrics
loss_object = tf.keras.losses.BinaryCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.SGD()

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.BinaryCrossentropy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.BinaryCrossentropy(name='test_accuracy')

In [31]:
# Training using GradientTape
@tf.function
def train_step(graphs, labels):
    with tf.GradientTape as tape:
        predictions = model(graphs)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    train_loss(loss)
    train_accuracy(labels, predictions)

# Perform training
@tf.function
def test_step(graphs, labels):
    predictions = model(graphs)
    t_loss = loss_object(graphs, labels)
    
    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [33]:
EPOCHS = 5

for epoch in range(EPOCHS):
    # Reset the metrics at the start of the next epoch
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()
    
    for graphs, labels in train_ds:
        train_step(graphs, labels)
    
    for test_graphs, test_labels in test_ds:
        test_step(test_graphs, test_labels)
    
    print(
        f'Epoch {epoch + 1}, '
        f'Loss: {train_loss.result()}, '
        f'Accuracy: {train_accuracy.result() * 100}, '
        f'Test Loss: {test_loss.result()}, '
        f'Test Accuracy: {test_accuracy.result() * 100}'
    )
        

tf.Tensor(
[[[0.]
  [0.]
  [0.]
  ...
  [0.]
  [0.]
  [0.]]

 [[0.]
  [0.]
  [0.]
  ...
  [0.]
  [0.]
  [0.]]

 [[0.]
  [0.]
  [0.]
  ...
  [0.]
  [0.]
  [0.]]

 ...

 [[0.]
  [0.]
  [0.]
  ...
  [0.]
  [0.]
  [0.]]

 [[0.]
  [0.]
  [0.]
  ...
  [0.]
  [0.]
  [0.]]

 [[0.]
  [0.]
  [0.]
  ...
  [0.]
  [0.]
  [0.]]], shape=(1024, 784, 1), dtype=float32)


AttributeError: in user code:

    <ipython-input-31-59632c5241b6>:4 train_step  *
        with tf.GradientTape as tape:

    AttributeError: __enter__


In [ ]:
# Testing
